# Задание 1 (10 баллов)

Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне.
Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)
Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.
GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).
Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции.
Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.


In [1]:
!pip install -q trl peft datasets -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip install -q -U huggingface_hub

In [2]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer, SFTConfig, SFTTrainer

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
# from trl import DPOTrainer
from tqdm.notebook import tqdm
import pandas as pd
import torch
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import ast
import re
import numpy as np

### Подготовка

In [3]:
system_prompt = """
You task is to write code in python according to user's instructions. Your answer (code) must be wrapped into ```*your code here* ```</s>.
"""

def chatml_format(example):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example['prompt']}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {
        "text": prompt
    }

In [4]:
dataset = load_dataset("notbadai/python_functions_reasoning", split='train')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/414M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/206299 [00:00<?, ? examples/s]

In [5]:
# sft_ds = dataset.select(i for i in range(100))
grpo_ds = dataset.select(i for i in range(200, 400))
test_ds = dataset.select(i for i in range(400, 1200))

Parameter 'indices'=<generator object <genexpr> at 0x79593c8fa400> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [6]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
device="cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "left"

# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

### GPRO

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Нам не понадобятся labels, так как мы будем проверять, работает ли код, а не его идентичность с кодом в ответах.

In [8]:
def format_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        start = re.search(r'```python', response)
        end = re.search(r'```</s>', response)
        if start is not None and end is not None:
          rewards.append(1.0)
        else:
          rewards.append(0.0)

    return rewards

In [9]:
def answer_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        try:
          matched = re.search(r'```python(.*?)```</s>', response, re.DOTALL)
          code = matched.group(1).strip()
          ast.parse(code)
          rewards.append(1.0)
        except Exception:
          rewards.append(0.0)

    return rewards

In [10]:
grpo_ds = grpo_ds.map(chatml_format)
test_ds =test_ds.map(chatml_format)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [11]:
batch = grpo_ds[2:3]

In [12]:
input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

output = model.generate(
    input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
    max_new_tokens=400, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
)

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [13]:
completions = tokenizer.batch_decode(output, skip_special_tokens=False)

In [14]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [15]:
training_args = GRPOConfig(output_dir="Qwen/Qwen2-0.5B-Instruct-python-sft-grpo",
                          #  per_device_train_batch_size=1,
                           logging_steps=5,
                           report_to="none",
                           num_generations=2,
                           num_train_epochs=1,
                           temperature=1.5,
                           label_names=["labels"])

In [16]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[answer_accuracy_func, format_accuracy_func],
    args=training_args,
    train_dataset=grpo_ds,
    peft_config=peft_config

)

In [17]:
trainer.train()

Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


TrainOutput(global_step=50, training_loss=0.0, metrics={'train_runtime': 1503.9378, 'train_samples_per_second': 0.133, 'train_steps_per_second': 0.033, 'total_flos': 0.0, 'train_loss': 0.0})

In [18]:
trainer.save_model('Qwen/Qwen2-0.5B-Instruct-python-sft-grpo')

### Оценка

In [19]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

original_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct", device_map=device)

In [20]:
def get_results(model, dataset, batch_size=2):
  device = model.device
  response = []
  dataset.set_format("pandas")
  for i in tqdm(range(0, 100, batch_size)):
      batch = dataset[i:i+batch_size]
      input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

      output = model.generate(
          input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
          max_new_tokens=512, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
      )[:,input_ids['input_ids'].shape[-1]:]
      response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))
  return response

In [ ]:
# result_df = pd.DataFrame({'prompt': dataset['prompt'][:100], 'answer': dataset['answer'][:100], 'response (before)': response_ref, 'response (after)': response})

In [21]:
original = get_results(original_model, test_ds)

  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [22]:
gpro = get_results(model, test_ds)

  0%|          | 0/50 [00:00<?, ?it/s]

In [23]:
result_df = pd.DataFrame({'prompt': test_ds['prompt'][:100], 'answer': test_ds['answer'][:100], 'response (before)': original, 'response (after)': gpro})

In [24]:
def parse_answer(text):
    m = re.search('```python(.*?)```</s>', text, re.DOTALL)
    if m is not None:
        return m.group(1)
    else:
        return ""

In [25]:
result_df['answer_before'] = result_df["response (before)"].apply(parse_answer)
result_df['answer_after'] = result_df["response (after)"].apply(parse_answer)
result_df['answer_correct'] = result_df["answer"].apply(parse_answer)

In [26]:
np.mean(format_accuracy_func(result_df['response (before)']))

np.float64(0.0)

In [27]:
np.mean(answer_accuracy_func(result_df["response (before)"]))

np.float64(0.0)

In [28]:
np.mean(format_accuracy_func(result_df['response (after)']))

np.float64(0.0)

In [29]:
np.mean(answer_accuracy_func(result_df['response (after)']))

np.float64(0.0)

In [30]:
result_df.head()

,prompt,answer,response (before),response (after),answer_before,answer_after,answer_correct
0,Remove elements from a list that have a specif...,"```python\ndef remove_elements(lst, target_sum...",The input list will contain positive integers...,"For example, given the list [123456789, 10000...",,,"\ndef remove_elements(lst, target_sum):\n d..."
1,"```python\ndef are_words_close(word1, word2):\...",```python\n if set(word1) != set(word2):\n ...,python\n\n# Importing necessary package for de...,,,,\n if set(word1) != set(word2):\n re...
2,"```python\ndef get_metadata(lst, row_name, idx...",```python\n for line in lst:\n if li...,python\n\n# Import necessary packages (if any ...,"\n\n\ndef get_metadata(lst, row_name, idx):\n ...",,,\n for line in lst:\n if line.starts...
3,"```python\ndef min_elements(nums: list, limit:...",```python\n current_sum = sum(nums)\n di...,python\n\n# Importing necessary package for de...,"\n\n\n```python\ndef min_elements(nums: list, ...",,,\n current_sum = sum(nums)\n difference ...
4,Write a function to sort a list of lists in as...,```python\ndef list_sum_sort(lst):\n return...,"Additionally, you need to handle cases where ...","Additionally, if there are multiple occurrenc...",,,\ndef list_sum_sort(lst):\n return sorted(l...
